# 🤖 Context-Aware RAG Chatbot
### Using LangChain + FAISS + Groq + Streamlit

| Component | Technology |
|-----------|------------|
| **Orchestration** | LangChain |
| **Embeddings** | `sentence-transformers/all-MiniLM-L6-v2` (local, free) |
| **Vector Store** | FAISS (local, fast) |
| **LLM** | Groq API — Llama 3 (free tier) |
| **Memory** | `ConversationBufferWindowMemory` (last 6 turns) |
| **UI** | Streamlit |
| **Knowledge Base** | 5 custom topic documents (~5K words) |

**Architecture:**
```
User Query → Embed Query → FAISS Search → Top-K Docs
                                              ↓
           Conversation Memory → Build Prompt → Groq LLM → Answer + Sources
```

## Step 1: Install Dependencies
⏱️ *Takes ~2 minutes on first run*

In [1]:
%%capture
!pip install langchain langchain-community langchain-core faiss-cpu \
             sentence-transformers groq streamlit python-dotenv \
             pyngrok -q
print('✅ All packages installed!')

In [4]:
# Verify imports
import langchain 
import faiss 
import sentence_transformers 
import groq
import streamlit
print(f'langchain         : {langchain.__version__}')
print(f'sentence-transformers: {sentence_transformers.__version__}')
print(f'groq              : {groq.__version__}')
print(f'streamlit         : {streamlit.__version__}')
print('✅ All imports verified!')

ImportError: cannot import name 'ComplexWarning' from 'numpy.core.numeric' (C:\Users\Uzair\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\numeric.py)

## Step 2: Set Up Project Files

In [ ]:
import os
os.makedirs('knowledge_base', exist_ok=True)
print('✅ Project directories created')

In [ ]:
%%writefile knowledge_base/artificial_intelligence.txt
ARTIFICIAL INTELLIGENCE — KNOWLEDGE BASE
=========================================

DEFINITION AND OVERVIEW
------------------------
Artificial Intelligence (AI) is the simulation of human intelligence processes by computer systems.
These processes include learning, reasoning, and self-correction. AI is broadly categorized into
Narrow AI (ANI), General AI (AGI), and Superintelligence (ASI).

HISTORY OF AI
-------------
AI as a formal field began in 1956 at the Dartmouth Conference, proposed by John McCarthy and others.
The modern AI renaissance began around 2012 with deep learning breakthroughs (AlexNet winning ImageNet).

KEY AI SUBFIELDS
----------------
Machine Learning: Systems that learn from data without explicit programming.
Natural Language Processing: Understanding and generating human language.
Computer Vision: Interpreting visual information from images and video.
Robotics: Physical AI agents that interact with the real world.
Expert Systems: AI that emulates decision-making of human experts.

MODERN AI ACHIEVEMENTS
----------------------
Deep Blue defeated chess champion Kasparov in 1997. AlphaGo defeated Go champion Lee Sedol in 2016.
GPT-3 (2020) and ChatGPT (2022) demonstrated remarkable language capabilities at scale.
GPT-4 (2023) achieved near-human performance on professional exams.
AlphaFold solved the protein structure prediction problem (2020).

AI ETHICS AND SAFETY
--------------------
Key concerns: algorithmic bias, privacy, accountability, job displacement, and existential risk.
The EU AI Act (2024) is the world's first comprehensive AI regulation framework.
Organizations like Anthropic and DeepMind have active AI safety research programs.

In [ ]:
%%writefile knowledge_base/machine_learning.txt
MACHINE LEARNING — KNOWLEDGE BASE
==================================

DEFINITION
----------
Machine Learning enables systems to learn from data and improve without being explicitly programmed.
Given enough examples, algorithms identify patterns and make decisions with minimal human intervention.

THREE MAIN PARADIGMS
--------------------
1. SUPERVISED LEARNING: Learns from labeled training data (input-output pairs).
   Examples: classification (spam detection), regression (house price prediction).
   Algorithms: Linear Regression, Decision Trees, Random Forests, SVM, Neural Networks.

2. UNSUPERVISED LEARNING: Finds hidden patterns in unlabeled data.
   Examples: customer segmentation, anomaly detection, dimensionality reduction.
   Algorithms: K-Means, DBSCAN, PCA, Autoencoders, GANs.

3. REINFORCEMENT LEARNING: Agent learns by interacting with environment for rewards/penalties.
   Examples: game playing (AlphaGo), robotics, recommendation systems.
   Algorithms: Q-Learning, DQN, Proximal Policy Optimization (PPO).

DEEP LEARNING
-------------
Deep Learning uses multi-layer neural networks to learn hierarchical representations.
CNNs excel at image data. RNNs/LSTMs handle sequential data. Transformers dominate NLP.

ML WORKFLOW
-----------
Problem Definition → Data Collection → EDA → Feature Engineering → Model Selection
→ Training → Evaluation → Hyperparameter Tuning → Deployment → Monitoring

COMMON PROBLEMS
---------------
Overfitting: Model memorizes training data; fix with regularization, dropout, more data.
Underfitting: Model too simple; fix with more complex model or better features.
Data Imbalance: One class dominates; fix with SMOTE, class weights.

EVALUATION METRICS
------------------
Classification: Accuracy, Precision, Recall, F1-Score, ROC-AUC.
Regression: MAE, MSE, RMSE, R-squared.
Frameworks: scikit-learn, PyTorch, TensorFlow, XGBoost, Hugging Face.

In [ ]:
%%writefile knowledge_base/llm_and_rag.txt
LARGE LANGUAGE MODELS & RAG — KNOWLEDGE BASE
=============================================

LARGE LANGUAGE MODELS (LLMs)
-----------------------------
LLMs are neural networks trained on massive text datasets to understand and generate language.
Based on the Transformer architecture (Vaswani et al., 2017). Trained as next-token predictors.
Modern LLMs: GPT-4, Claude 3, Gemini, Llama 3, Mistral, Mixtral.

TRAINING TECHNIQUES
-------------------
Pre-training: Self-supervised learning on massive unlabeled text corpora.
Fine-tuning: Supervised adaptation on task-specific labeled data.
RLHF: Reinforcement Learning from Human Feedback — aligns to human preferences.
Constitutional AI: Anthropic's method using self-critique and revision.

RETRIEVAL-AUGMENTED GENERATION (RAG)
--------------------------------------
RAG augments LLMs with external knowledge retrieval at inference time.
Instead of relying only on parametric memory (weights), RAG retrieves relevant documents
and injects them as context into the LLM prompt.

RAG PIPELINE STEPS:
1. Document Ingestion: Load docs → chunk → embed → store in vector DB
2. Retrieval: Embed query → similarity search → return top-K chunks
3. Augmentation: Inject retrieved context into LLM prompt
4. Generation: LLM produces answer grounded in retrieved facts

WHY RAG MATTERS
---------------
Reduces hallucination by grounding responses in retrieved facts.
Enables real-time knowledge without model retraining.
Supports source citations — users can verify information.
Cost-effective alternative to full fine-tuning for domain adaptation.

VECTOR DATABASES
----------------
FAISS: Fast local vector search (Meta, open source) — ideal for development.
Pinecone: Managed cloud vector DB — production ready.
Chroma: Simple Python-native, great for prototyping.
Weaviate, Qdrant, Milvus: Alternative open-source options.

EMBEDDING MODELS
----------------
all-MiniLM-L6-v2: Fast, 384-dim, excellent for RAG (free, local).
text-embedding-ada-002: High quality, OpenAI API-based.
BGE-large-en: State-of-the-art open embedding model.

RAG EVALUATION (RAGAS)
----------------------
Context Precision: Are retrieved chunks relevant?
Context Recall: Are all relevant chunks retrieved?
Faithfulness: Is the answer grounded in the context?
Answer Relevance: Does the answer address the question?

In [ ]:
%%writefile knowledge_base/prompt_engineering.txt
PROMPT ENGINEERING — KNOWLEDGE BASE
=====================================

DEFINITION
----------
Prompt engineering is designing and optimizing inputs to guide LLMs toward desired outputs.
It is a critical skill for effectively using LLMs without modifying model weights.

CORE TECHNIQUES
---------------
ZERO-SHOT: Task instruction only, no examples. Best for simple, well-defined tasks.
FEW-SHOT: 2-10 input/output examples before the query. Best for format-specific tasks.
CHAIN-OF-THOUGHT (CoT): "Let's think step by step" — improves multi-step reasoning.
ROLE PROMPTING: "You are an expert X" — activates relevant knowledge associations.
STRUCTURED OUTPUT: Request JSON/XML format — ensures parseable, consistent responses.

PROMPT COMPONENTS
-----------------
Role/Persona: Who is the AI in this context?
Context/Background: Relevant information to inform the response.
Task/Instruction: What specifically should the model do?
Input Data: The actual content to process.
Output Format: How should the response be structured?
Constraints: What should/shouldn't be included?
Examples: Few-shot demonstrations.

ADVANCED TECHNIQUES
-------------------
ReAct: Interleave Reasoning + Acting (tool calls).
Tree of Thoughts: Explore multiple reasoning paths and evaluate intermediates.
Self-Consistency: Sample multiple responses and take majority answer.
DSPy: Programmatic prompt optimization using automatic compilation.

ANTI-PATTERNS TO AVOID
----------------------
Vague instructions, contradictory constraints, ambiguous pronouns,
overloaded prompts, missing output format specification.

In [ ]:
%%writefile knowledge_base/python_programming.txt
PYTHON PROGRAMMING — KNOWLEDGE BASE
=====================================

OVERVIEW
--------
Python is a high-level, interpreted, general-purpose programming language created by Guido van Rossum (1991).
It emphasizes readability and supports procedural, OOP, and functional paradigms.
Most popular language globally; dominates data science, ML, and AI development.

KEY FEATURES
------------
Dynamic typing, garbage collection, first-class functions, list/dict comprehensions,
context managers (with), decorators, generators, and optional type hints (PEP 484).

DATA STRUCTURES
---------------
List: Ordered, mutable. | Tuple: Ordered, immutable. | Dict: Key-value mapping.
Set: Unique elements. | Deque: Double-ended queue, O(1) appends.

DATA SCIENCE ECOSYSTEM
----------------------
NumPy: N-dimensional arrays and numerical computing.
Pandas: DataFrames for data manipulation and analysis.
Matplotlib/Seaborn: Visualization libraries.
Scikit-learn: Machine learning algorithms.
PyTorch/TensorFlow: Deep learning frameworks.

BEST PRACTICES
--------------
Follow PEP 8 style guide. Use virtual environments (venv/conda).
Write docstrings. Use type hints. Write pytest unit tests.
Use f-strings (Python 3.6+). Prefer pathlib over os.path.

WEB FRAMEWORKS
--------------
FastAPI: Modern async REST API framework — fastest Python web framework.
Django: Full-stack web framework with batteries included.
Flask: Lightweight microframework for simple web apps.
Streamlit: Rapid data app development — ideal for ML demos.
Gradio: Simple ML model interfaces.

## Step 3: Create the RAG Pipeline Module

In [ ]:
%%writefile rag_pipeline.py
"""
RAG Pipeline: LangChain + FAISS + HuggingFace Embeddings + Groq LLM
"""

import os
from pathlib import Path
from typing import List, Dict, Any

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from groq import Groq

KB_DIR = Path("knowledge_base")


class RAGPipeline:
    SYSTEM_PROMPT = """You are a knowledgeable AI assistant with access to a curated knowledge base.
Use the retrieved context to answer questions accurately. Always cite which document your answer
comes from. If context is insufficient, say so and answer from general knowledge."""

    PROMPT_TEMPLATE = """
[CONVERSATION HISTORY]
{history}

[RETRIEVED CONTEXT]
{context}

[CURRENT QUESTION]
{question}

Answer using the retrieved context and conversation history. Be specific and cite sources:"""

    def __init__(self, api_key: str):
        self.groq_client = Groq(api_key=api_key)
        self.memory = ConversationBufferWindowMemory(
            k=6, return_messages=False,
            human_prefix="Human", ai_prefix="Assistant"
        )
        self.prompt = PromptTemplate(
            input_variables=["history", "context", "question"],
            template=self.PROMPT_TEMPLATE
        )
        self.vectorstore = None
        self._build_vectorstore()

    def _load_docs(self) -> List[Document]:
        docs = []
        for fp in KB_DIR.glob("*.txt"):
            text = fp.read_text(encoding="utf-8")
            docs.append(Document(
                page_content=text,
                metadata={"source": fp.stem.replace("_", " ").title(), "file": fp.name}
            ))
            print(f"  ✓ Loaded: {fp.name} ({len(text)} chars)")
        return docs

    def _build_vectorstore(self):
        print("📚 Loading knowledge base...")
        docs = self._load_docs()
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=60)
        chunks = splitter.split_documents(docs)
        print(f"✂️  {len(chunks)} chunks created")

        print("🔢 Generating embeddings...")
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={"device": "cpu"},
            encode_kwargs={"normalize_embeddings": True},
        )
        self.vectorstore = FAISS.from_documents(chunks, embeddings)
        print("✅ Vector store ready!")

    def retrieve(self, query: str, top_k: int = 3) -> List[Document]:
        return self.vectorstore.similarity_search(query, k=top_k)

    def chat(self, question: str, top_k: int = 3,
             model: str = "llama3-8b-8192", **kwargs) -> Dict[str, Any]:
        # Retrieve
        docs = self.retrieve(question, top_k=top_k)

        # Format context
        context_parts, sources = [], []
        seen = set()
        for doc in docs:
            src = doc.metadata.get("source", "Unknown")
            context_parts.append(f"[{src}]\n{doc.page_content}")
            if src not in seen:
                sources.append(src)
                seen.add(src)
        context_str = "\n\n---\n\n".join(context_parts) or "No context available."

        # Get history
        history = self.memory.load_memory_variables({}).get("history", "No prior conversation.")

        # Build prompt & call LLM
        filled = self.prompt.format(history=history, context=context_str, question=question)
        response = self.groq_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": self.SYSTEM_PROMPT},
                {"role": "user",   "content": filled},
            ],
            temperature=0.4,
            max_tokens=800,
        )
        answer = response.choices[0].message.content.strip()

        # Update memory
        self.memory.save_context({"input": question}, {"output": answer})

        return {"answer": answer, "sources": sources, "contexts": docs}

    def clear_memory(self): self.memory.clear()


def build_pipeline(api_key: str) -> RAGPipeline:
    return RAGPipeline(api_key=api_key)

## Step 4: Test RAG Pipeline in Notebook (No UI)

Run this cell to verify the pipeline works before launching Streamlit.

In [ ]:
# ── Enter your Groq API key ──────────────────────────────────────────────────
# Free key: https://console.groq.com  (sign up → API Keys → Create key)
GROQ_API_KEY = "gsk_your_key_here"   # ← REPLACE WITH YOUR KEY

# Build pipeline
from rag_pipeline import build_pipeline
pipeline = build_pipeline(GROQ_API_KEY)

print("\n" + "="*60)
print("✅ RAG Pipeline Ready! Running test queries...")
print("="*60)

In [ ]:
# ── Test Query 1: RAG ────────────────────────────────────────────────────────
q1 = "What is Retrieval-Augmented Generation and why is it important?"
result1 = pipeline.chat(q1, top_k=3)

print(f"❓ Question : {q1}")
print(f"📚 Sources  : {', '.join(result1['sources'])}")
print(f"🤖 Answer   :\n{result1['answer']}")

In [ ]:
# ── Test Query 2: Follow-up (tests MEMORY) ──────────────────────────────────
q2 = "What vector databases would you recommend for it?"
result2 = pipeline.chat(q2, top_k=3)

print(f"❓ Question : {q2}")
print(f"📚 Sources  : {', '.join(result2['sources'])}")
print(f"🤖 Answer   :\n{result2['answer']}")
print("\n💡 Note: The answer references the previous question context (memory working!)")

In [ ]:
# ── Test Query 3: Different topic ──────────────────────────────────────────
q3 = "Explain the difference between supervised and unsupervised learning with examples."
result3 = pipeline.chat(q3, top_k=3)

print(f"❓ Question : {q3}")
print(f"📚 Sources  : {', '.join(result3['sources'])}")
print(f"🤖 Answer   :\n{result3['answer']}")

In [ ]:
# ── Show Conversation Memory ─────────────────────────────────────────────────
history = pipeline.memory.load_memory_variables({})["history"]
print("📖 CONVERSATION HISTORY (stored in memory):")
print("=" * 60)
print(history[:1500] + "..." if len(history) > 1500 else history)

In [ ]:
# ── Retrieval Visualization ──────────────────────────────────────────────────
import matplotlib.pyplot as plt
import numpy as np

# Show similarity scores for a test query
test_query = "How do vector databases work for semantic search?"
retrieved = pipeline.vectorstore.similarity_search_with_score(test_query, k=5)

labels = [f"{doc.metadata['source']}\n({doc.page_content[:30]}...)" for doc, _ in retrieved]
scores = [1 - score for _, score in retrieved]  # Convert distance to similarity

fig, ax = plt.subplots(figsize=(10, 4))
colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(scores)))
bars = ax.barh(range(len(labels)), scores, color=colors, edgecolor='black', linewidth=0.5)
ax.set_yticks(range(len(labels)))
ax.set_yticklabels(labels, fontsize=9)
ax.set_xlabel('Cosine Similarity Score')
ax.set_title(f'Retrieved Documents for:\n"{test_query}"', fontweight='bold')
for bar, score in zip(bars, scores):
    ax.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
            f'{score:.3f}', va='center', fontsize=10)
ax.set_xlim(0, 1.1)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('retrieval_scores.png', dpi=130, bbox_inches='tight')
plt.show()
print('✅ Retrieval visualization saved!')

## Step 5: Create Streamlit App File

In [ ]:
%%writefile app.py
"""Context-Aware RAG Chatbot — Streamlit UI"""

import streamlit as st
import os

st.set_page_config(page_title="RAG Chatbot", page_icon="🤖", layout="wide")

st.markdown("""
<style>
.user-msg { background: #1e3a5f; color: white; padding: 12px 16px;
           border-radius: 15px 15px 3px 15px; margin: 5px 0; max-width: 75%; margin-left: auto; }
.bot-msg  { background: #1a1a2e; color: #e8e8f0; padding: 12px 16px;
           border-radius: 15px 15px 15px 3px; margin: 5px 0; max-width: 80%;
           border: 1px solid #3a3a6a; }
.src-badge { display:inline-block; background: rgba(100,120,255,0.2); color: #8888ff;
             border: 1px solid #5555cc; border-radius: 10px; padding: 2px 8px;
             font-size: 11px; margin: 2px; }
</style>
""", unsafe_allow_html=True)

@st.cache_resource(show_spinner=False)
def load_pipeline(api_key):
    from rag_pipeline import build_pipeline
    return build_pipeline(api_key)

# Session state
if "messages" not in st.session_state:
    st.session_state.messages = []

# ── Sidebar ─────────────────────────────────────────────────────────────────
with st.sidebar:
    st.title("🤖 RAG Chatbot")
    st.caption("LangChain · FAISS · Groq · Context Memory")
    st.divider()

    api_key = st.text_input("🔑 Groq API Key", type="password",
                            placeholder="gsk_...", value=os.getenv("GROQ_API_KEY", ""))
    model = st.selectbox("🧠 Model",
        ["llama3-8b-8192", "llama3-70b-8192", "mixtral-8x7b-32768", "gemma2-9b-it"])
    top_k = st.slider("📚 Top-K Docs", 1, 6, 3)
    show_src = st.toggle("🔍 Show Sources", True)

    if st.button("🚀 Initialize", use_container_width=True, type="primary"):
        if not api_key:
            st.error("Enter your API key first.")
        else:
            with st.spinner("Building RAG pipeline (~30s on first run)..."):
                try:
                    p = load_pipeline(api_key)
                    st.session_state.pipeline = p
                    st.session_state.ready = True
                    st.success("✅ Ready!")
                except Exception as e:
                    st.error(f"Error: {e}")

    st.divider()
    st.markdown("**📚 Knowledge Base:**")
    st.markdown("- 🤖 Artificial Intelligence\n- 🧬 Machine Learning\n- 🌐 LLMs & RAG\n- 💡 Prompt Engineering\n- 🐍 Python")

    if st.button("🗑️ Clear Chat", use_container_width=True):
        st.session_state.messages = []
        if hasattr(st.session_state, 'pipeline'):
            st.session_state.pipeline.clear_memory()
        st.rerun()

    if st.session_state.get('ready'):
        st.metric("💬 Turns", len(st.session_state.messages) // 2)

# ── Main ─────────────────────────────────────────────────────────────────────
st.title("💬 Context-Aware RAG Chatbot")
st.caption("Retrieval-Augmented Generation with Conversational Memory")

if not st.session_state.get('ready'):
    st.info("👈 Enter your Groq API key in the sidebar and click **Initialize**.")
    st.markdown("""
    ### 💡 Try asking:
    - *"What is RAG and how does it work?"*
    - *"Explain the transformer architecture"*
    - *"What's the difference between supervised and unsupervised learning?"*
    - *"How do vector databases enable semantic search?"*
    - *"What are the best practices for prompt engineering?"*
    """)

for msg in st.session_state.messages:
    if msg["role"] == "user":
        st.markdown(f'<div class="user-msg">👤 {msg["content"]}</div>', unsafe_allow_html=True)
    else:
        st.markdown(f'<div class="bot-msg">🤖 {msg["content"]}</div>', unsafe_allow_html=True)
        if show_src and msg.get("sources"):
            badges = " ".join(f'<span class="src-badge">📄 {s}</span>' for s in msg["sources"])
            st.markdown(f"Sources: {badges}", unsafe_allow_html=True)

if prompt := st.chat_input("Ask a question...", disabled=not st.session_state.get('ready')):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.spinner("🔍 Retrieving & generating..."):
        try:
            res = st.session_state.pipeline.chat(prompt, top_k=top_k, model=model)
            st.session_state.messages.append({
                "role": "assistant", "content": res["answer"], "sources": res["sources"]
            })
        except Exception as e:
            st.session_state.messages.append({"role": "assistant",
                "content": f"Error: {e}", "sources": []})
    st.rerun()

## Step 6: Launch Streamlit in Colab via ngrok

In [ ]:
# ── Method 1: Using localtunnel (no account needed) ────────────────────────
import subprocess, threading, time

# Start Streamlit
proc = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
time.sleep(5)

# Use colab's port forwarding
try:
    from google.colab.output import eval_js
    url = eval_js("google.colab.kernel.proxyPort(8501)")
    print(f"\n🌐 Streamlit App URL: {url}")
    print("  → Click the URL above to open the chatbot!")
except:
    print("🌐 Streamlit running on port 8501")
    print("   If using locally: http://localhost:8501")

print("\n💡 Tip: Enter your Groq API key in the sidebar to start chatting!")

In [ ]:
# ── Method 2: Using ngrok (optional, needs free account) ───────────────────
# 1. Sign up free at https://ngrok.com
# 2. Copy your authtoken from https://dashboard.ngrok.com/auth

# NGROK_TOKEN = "your_ngrok_token_here"   # ← REPLACE

# from pyngrok import ngrok
# ngrok.set_auth_token(NGROK_TOKEN)
# public_url = ngrok.connect(8501)
# print(f"🌐 Public URL: {public_url}")
# print("Share this link with anyone to demo your chatbot!")

print('ℹ️  ngrok method commented out — uncomment to get a public shareable URL')

## 📊 Step 7: RAG System Analysis

Evaluate the system's retrieval performance and visualize key metrics.

In [ ]:
# ── Retrieval Quality Analysis ──────────────────────────────────────────────
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

# Test queries covering different topics
test_queries = [
    ("What is RAG and how does FAISS work?",          "Llm And Rag"),
    ("Explain overfitting and how to fix it",          "Machine Learning"),
    ("What is few-shot prompting?",                    "Prompt Engineering"),
    ("History of artificial intelligence",            "Artificial Intelligence"),
    ("Python best practices and virtual environments", "Python Programming"),
]

# Check if top-1 retrieval matches expected source
results = []
for query, expected_src in test_queries:
    retrieved = pipeline.retrieve(query, top_k=3)
    sources = [d.metadata['source'] for d in retrieved]
    hit = expected_src in sources
    rank = sources.index(expected_src) + 1 if hit else None
    results.append({"query": query[:45]+"...", "expected": expected_src,
                    "got": sources[:3], "hit": hit, "rank": rank})

# Compute MRR (Mean Reciprocal Rank)
mrr = np.mean([1/r['rank'] if r['rank'] else 0 for r in results])
recall_at_3 = np.mean([r['hit'] for r in results])

print("📊 RETRIEVAL EVALUATION")
print("=" * 65)
for r in results:
    status = f"✅ Rank {r['rank']}" if r['hit'] else "❌ Not found"
    print(f"  {status} | {r['query']}")
print("=" * 65)
print(f"  Recall@3        : {recall_at_3:.0%}")
print(f"  Mean Reciprocal Rank (MRR): {mrr:.3f}")

In [ ]:
# ── Architecture Diagram ─────────────────────────────────────────────────────
import matplotlib.patches as mpatches
import matplotlib.patheffects as pe

fig, ax = plt.subplots(figsize=(14, 6))
ax.set_xlim(0, 14)
ax.set_ylim(0, 6)
ax.axis('off')
ax.set_facecolor('#0d0d1a')
fig.patch.set_facecolor('#0d0d1a')
ax.set_title('RAG Chatbot Architecture', color='white', fontsize=16, fontweight='bold', pad=10)

# Define boxes
components = [
    (0.3, 2.5, 1.8, 1.2, '#1e3a5f', '👤 User\nQuery',         'white'),
    (2.5, 2.5, 1.8, 1.2, '#2d1b4e', '🔢 Embed\nQuery',       '#c8a0ff'),
    (4.7, 2.5, 1.8, 1.2, '#1a3a2a', '🔍 FAISS\nSearch',      '#80ffaa'),
    (6.9, 3.8, 1.8, 1.2, '#3a2a00', '📚 Knowledge\nBase',    '#ffd080'),
    (6.9, 1.2, 1.8, 1.2, '#2a1a0a', '🧠 Memory\n(k=6 turns)','#ffb060'),
    (9.1, 2.5, 1.8, 1.2, '#1a1a3a', '📝 Prompt\nTemplate',   '#a0a8ff'),
    (11.3, 2.5, 1.8, 1.2,'#3a1a1a', '🤖 Groq LLM\n(Llama 3)','#ff8080'),
]

boxes = {}
for x, y, w, h, color, label, tc in components:
    rect = mpatches.FancyBboxPatch((x, y), w, h, boxstyle="round,pad=0.1",
                                    facecolor=color, edgecolor='#4444aa', linewidth=1.5)
    ax.add_patch(rect)
    ax.text(x + w/2, y + h/2, label, ha='center', va='center',
            color=tc, fontsize=9, fontweight='bold')

# Arrows
arrows = [
    (2.1, 3.1, 0.35, 0),   (4.3, 3.1, 0.35, 0),
    (6.5, 3.1, 0.35, 0.7), (6.5, 3.1, 0.35, -0.7),
    (8.7, 3.1, 0.35, 0),   (10.9, 3.1, 0.35, 0),
]
for x, y, dx, dy in arrows:
    ax.annotate('', xy=(x+dx, y+dy), xytext=(x, y),
                arrowprops=dict(arrowstyle='->', color='#6666cc', lw=1.8))

plt.tight_layout()
plt.savefig('architecture.png', dpi=130, bbox_inches='tight', facecolor='#0d0d1a')
plt.show()
print('✅ Architecture diagram saved!')

## ✅ Summary

| Component | Implementation |
|-----------|----------------|
| **RAG Framework** | LangChain (chains, memory, prompts) |
| **Embeddings** | `all-MiniLM-L6-v2` — 384-dim, free, local |
| **Vector Store** | FAISS — cosine similarity, no server needed |
| **Chunking** | `RecursiveCharacterTextSplitter` (500 tokens, 60 overlap) |
| **LLM** | Groq Llama 3 8B — fast free inference |
| **Memory** | `ConversationBufferWindowMemory` — last 6 turns |
| **UI** | Streamlit — dark theme, source citations, session stats |
| **Knowledge Base** | 5 topics × ~800 words = ~4000 words corpus |

**Skills Demonstrated:** RAG · Document Embedding · Vector Search · LangChain · Prompt Engineering · Streamlit Deployment · Conversational Memory